In [1]:
import numpy as np
import pickle
import cv2
import os

from vocab import Vocab, load_vocab
from common import Data, Split, Batches, load_data, encode_y, load_split
from pack import Pack

%load_ext autoreload
%autoreload 2

In [2]:
genre_list = pickle.load(open("../data/tmdb_genres_list.pkl", 'rb'))

GENRES = load_data("../local/genres.pkl")
train = load_split("../local/train.pkl")
val = load_split("../local/val.pkl")
test = load_split("../local/test.pkl")
OVERVIEWS = load_data("../local/overviews_encoded.pkl")

In [3]:
X_train = train.get_data(OVERVIEWS)
X_val = val.get_data(OVERVIEWS)
X_test = test.get_data(OVERVIEWS)
Y_train = train.get_data(GENRES)
Y_val = val.get_data(GENRES)
Y_test = test.get_data(GENRES)

X_train = [" ".join([str(e) for e in x]) for x in X_train]
X_val = [" ".join([str(e) for e in x]) for x in X_val]
X_test = [" ".join([str(e) for e in x]) for x in X_test]

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
cv = CountVectorizer()

In [5]:
X_train_cv = cv.fit_transform(X_train)
X_val_cv = cv.transform(X_val)
X_test_cv = cv.transform(X_test)

In [14]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import f1_score, roc_auc_score

from skmultilearn.problem_transform import BinaryRelevance

from sklearn.multiclass import OneVsRestClassifier

from sklearn.svm import LinearSVC, SVC

In [7]:
svm = OneVsRestClassifier(LinearSVC())
svm.fit(X_train_cv, Y_train)

OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [8]:
val_predict_svm = svm.predict(X_val_cv)
test_predict_svm = svm.predict(X_test_cv)

In [9]:
print(f1_score(Y_val, val_predict_svm, average="micro"))
#print(f1_score(Y_val, val_predict_svm, average="macro"))
#print(f1_score(Y_val, val_predict_svm, average="weighted"))

0.5013595752945745


In [10]:
print(f1_score(Y_test, test_predict_svm, average="micro"))
#print(f1_score(Y_test, test_predict_svm, average="macro"))
#print(f1_score(Y_test, test_predict_svm, average="weighted"))

0.49203213486105624


In [12]:
val_proba_svm = svm.decision_function(X_val_cv)

In [19]:
print(roc_auc_score(Y_val,val_proba_svm,average='micro'))
#print(roc_auc_score(Y_val,val_proba_svm,average='macro'))
#print(roc_auc_score(Y_val,val_proba_svm,average='weighted'))

0.8460884411504801
